In [126]:
import numpy as np
import pandas as pd
import random

In [127]:
VM_catalogue = pd.read_excel("VM_Catalogue.xlsx",skiprows=1)
workload = pd.read_excel("Workload.xlsx")

CPU_VM = VM_catalogue["CPU"].values
RAM_VM = VM_catalogue["RAM"].values
COUT_VM = VM_catalogue["Cost"].values
DISPO_VM = VM_catalogue["Quantity"].values
print(CPU_VM)

ReqCPU = np.array([1, 1, 1, 1])
ReqRAM = np.array([0.25, 0.25, 0.25, 0.25])
mu = np.array([1416, 696, 1005, 1259])

# λ_i = max de la trace
lamb = workload.max()["input_rate"]
print(lamb)

[  1   2   2   2   4   4   4   8   8   8  16  16  16  40  80  96 160]
5751


- We read our data stored in VM_Catalogue.xlsx and workload.xlsx

- VM_Catalogue.xlsx is read while ignoring the first row (skiprows=1) and contains the list of virtual machines

- workload contains load traces (input_rate, CPU load, RAM load…)

- We displayed the values of the CPU column from the file VM_Catalogue.xlsx. There are 17 types of VMs with the following CPU capacities [1 2 2... 96 160]

We retrieve the columns from the catalogue into arrays:

- CPU_VM corresponds to the number of vCPUs per VM type

- RAM_VM corresponds to the amount of RAM

- COUT_VM corresponds to the cost per VM

- DISPO_VM corresponds to the number of available machines

Our application is made up of 4 components. We then define
- ReqCPU as the CPU required to run an instance of component

- ReqRAM as the RAM required for an instance of component
- µ represents the processing capacity of an instance of the component

We chose to keep it the maximum value of the input_rate column from your trace (5751) because if it works for the max, the code works for lower values.

In [128]:
NB_COMP = len(ReqCPU)
NB_VM = len(CPU_VM)
print(NB_VM)
def replicas_minimales():
    return np.ceil(lamb / mu).astype(int)

17


- The function is used to calculate the minimum number of instances of each component needed to handle the load.

- To prevent the component from saturating, we need: r_i > λ / µ
. So the minimum number of instances = ceil(λ / µ)

In [129]:
def lambda_scaled(k):
    return k * lamb

- lambda_scaled(k) returns the load λ multiplied by a factor k to simulate different load levels

In [130]:
def replicas_minimales_k(k):
    return np.ceil((k * lamb) / mu).astype(int)

- we calculate the minimum number of instances required for each component, but at a load λ multiplied by a factor k

The higher k increases:
- the load increases
- so more irrrrrrrto avoid overloading the components

In [131]:
def greedy_placement(r, x):

    y = np.zeros((NB_COMP, NB_VM), dtype=int)  #empty placement matrix
    vm_order = np.argsort(COUT_VM)  #table showing the indices of the VMs sorted by cost

    for i in range(NB_COMP):
        remaining  = r[i]
        for j in vm_order:
            cpu_util = np.dot(y[:, j], ReqCPU)
            ram_util = np.dot(y[:, j], ReqRAM)

            max_cpu = (x[j]*CPU_VM[j] - cpu_util) // ReqCPU[i]
            max_ram = (x[j]*RAM_VM[j] - ram_util) // ReqRAM[i]
            max_possible = int(min(max_cpu, max_ram, remaining ))

            if max_possible > 0:
                y[i, j] += max_possible
                remaining  -= max_possible

            if remaining  == 0:
                break

        if remaining  > 0:
            return None

    return y


- The function attempts to place component instances on VM types using a greedy strategy.

- VM types are considered in increasing cost order (cheapest VMs first).

- For each component, the function tries to place as many instances as possible on each VM type while respecting CPU and RAM capacities.

- The placement verifies resource constraints: available CPU, available RAM, and available VM quantities.

- If all required instances of every component can be placed, the function returns the placement matrix y[i, j].

- If at least one component cannot be fully placed, the function returns None, indicating that the placement is impossible.

- The matrix y indicates how many instances of each component are assigned to each VM type.



In [132]:
def total_cost(x):
    return np.sum(x * COUT_VM)

def feasible(r, x):
    return greedy_placement(r, x)


total_cost(x)

- We calculate the total cost of a VM configuration.

- We multiply the number of VMs of each type (x[j]) by their unit cost (COUT_VM[j]).
- We return the sum of all VM costs.
- This is useful for evaluating or comparing deployment solutions.

feasible(r, x)

- This function checks if a given VM configuration x can host all required component replicas r.

- We call the greedy placement function to try to allocate all components on the VMs.

- We return True if the placement succeeds, False otherwise.

- This is used to quickly validate feasibility before evaluating cost or optimizing.

In [133]:
def repair(r, x):

    r = np.maximum(r, replicas_minimales())

    x = np.minimum(x, DISPO_VM)

    tests = 0
    while greedy_placement(r, x) is None and tests < 100:
        j_best = np.argmin(COUT_VM / CPU_VM)

        if x[j_best] < DISPO_VM[j_best]:
            x[j_best] += 1
        else:
            j_alt = random.randint(0, NB_VM-1)
            if x[j_alt] < DISPO_VM[j_alt]:
                x[j_alt] += 1
        tests += 1

    if greedy_placement(r, x) is None:
        return None, None

    return r, x


- We made sure to guarantee stability by enforcing a minimum number of replicas for each component.

- This code allows limiting the number of VMs used while respecting the quantities available in the catalog.

- We created a loop that gradually adds VMs as long as placement remains impossible.

- This code allows prioritizing the addition of the most cost-effective VM according to the cost/CPU ratio.

- We made a random selection of an alternative VM if the most cost-effective VM is no longer available.

- We repeated this process up to 100 attempts to obtain a valid solution.

- This code allows checking the final feasibility with greedy placement.

- We return the repaired solution (r, x) if placement is possible, otherwise (None, None).

In [134]:
def initial_solution():

    r = replicas_minimales().copy()

    j_best = np.argmin(COUT_VM / CPU_VM)
    x = np.zeros(NB_VM, dtype=int)

    cpu_total = np.sum(r * ReqCPU)
    x[j_best] = int(np.ceil(cpu_total / CPU_VM[j_best]))

    r2, x2 = repair(r, x)

    if r2 is None:
        x_random = np.zeros(NB_VM, dtype=int)
        for j in range(NB_VM):
            x_random[j] = random.randint(0, 3)

        r2, x2 = repair(r, x_random)

        tests = 0
        while r2 is None and tests < 50:
            x_random = np.random.randint(0, 3, size=NB_VM)
            r2, x2 = repair(r, x_random)
            tests += 1

        if r2 is None:
            raise RuntimeError("Unable to build a feasible initial solution.")

    return r2, x2


- We built an initial solution that is always feasible.

- We calculated the minimal replicas for each component to ensure stability.

- This code allows you to first choose the most cost-effective VM according to the cost/CPU ratio to provision sufficient resources.

- We calculated the number of VMs needed to cover the total CPU demand.

- This code allows you to attempt to repair this first solution with the repair() function to ensure it is valid.

- We planned an alternative strategy if the initial placement fails: generating a random but reasonable initialization.

- We repeated the random repair until a feasible solution was found, with a maximum number of attempts to avoid an infinite loop.

- This code allows an error to be raised if no valid initial solution can be found after all attempts.

- we return the solution (r2, x2) which is guaranteed to be feasible.

In [135]:
def neighbor(r, x):
    r2, x2 = r.copy(), x.copy()

    if random.random() < 0.5:
        i = random.randint(0, NB_COMP-1)
        r2[i] = max(1, r2[i] + random.choice([-1, 1]))
    else:
        j = random.randint(0, NB_VM-1)
        x2[j] = max(0, x2[j] + random.choice([-1, 1]))

    return r2, x2


This code aims to generate a neighboring solution from an existing solution (r, x)

- We create a copy of the vectors r and x so as not to modify the original solution

- We randomly choose to modify either a replica or the number of VMs

- To modify a replica: we select a random component and increase or decrease its number of instances by one, ensuring at least 1 replica.

- To modify the number of VMs: we select a random VM type and increase or decrease its number of instances by one, ensuring at least 0 VMs

- We return the new solution (r2, x2) which is a neighbor of the original solution.

In [136]:
def hill_climbing(nb_iter=2000):

    r, x = initial_solution()
    best_cost = total_cost(x)

    for _ in range(nb_iter):
        r2, x2 = neighbor(r, x)
        r2, x2 = repair(r2, x2)

        if r2 is None:
            continue

        new_cost = total_cost(x2)

        if new_cost < best_cost:
            r, x = r2, x2
            best_cost = new_cost

    y = greedy_placement(r, x)
    return r, x, y, best_cost


- This code aims to optimize the placement of components on VMs in order to minimize the total cost using a hill climbing approach.

- A feasible solution (r, x) is initialized with initial_solution().

- The cost of the initial solution is calculated with total_cost(x).

The process is repeated for nb_iter iterations:

- A neighboring solution (r2, x2) is generated from the current solution using neighbor().

- An attempt is made to repair this neighboring solution to ensure its feasibility with repair().

- If the neighboring solution is impossible (r2 is None), the process moves to the next iteration.

- The cost of the repaired neighboring solution is calculated.

- If the new cost is lower than the current best cost, this solution is adopted as the new best solution (r, x) and the best cost is updated.

After all iterations, the final placement of instances is calculated with greedy_placement(r, x) and the final solution (r, x), the placement matrix y, and the best cost are returned.

In [137]:
r_opt, x_opt, y_opt, c_opt = hill_climbing()

print("Optimal cost:", c_opt)
print("Optimal replicas:", r_opt)
print("Optimal VMs:", x_opt)
print("Placement y_ij:\n", y_opt)


Optimal cost: 1.7648
Optimal replicas: [5 9 6 5]
Optimal VMs: [0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0]
Placement y_ij:
 [[0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 9 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0]]


- This code is intended to run the hill climbing algorithm to find an optimal solution for placing components on the VMs

We run hill_climbing() and obtain:

- r_opt: the optimal number of replicas for each component,

- x_opt: the optimal number of VMs of each type,

- y_opt: the matrix of replica placement on the VMs,

- c_opt: the minimal total cost achieved.

ALGO GA

In [138]:
def random_individual():

    r = replicas_minimales().copy()

    for i in range(NB_COMP):
        if random.random() < 0.3:
            r[i] += random.randint(0, 2)

    x = np.zeros(NB_VM, dtype=int)
    for j in range(NB_VM):
        if random.random() < 0.3:
            x[j] = random.randint(0, 3)

    r, x = repair(r, x)
    if r is None:
        return random_individual()

    return {"r": r, "x": x, "Cost": total_cost(x)}


- This code is intended to generate a random solution (r, x) for optimization.

-  We initialize the replicas r with the minimum required number for each component (to ensure stability).

- We sometimes randomly add +1 or +2 replicas to certain components to introduce variation into the solution.

- We initialize the VM vector x to zero and randomly choose a few VMs (with a number between 0 and 3) to introduce diversity in the number of VMs used.

- We use the function repair(r, x) to ensure that the generated solution is feasible.

- If the repair fails, we recursively call the function to generate a new random solution.

Finally, we return a dictionary containing:

- "r": the number of replicas per component,

- "x": the number of VMs of each type,

- "cost": the total cost of the solution calculated with total_cost(x).

In [139]:
def selection(population, k=3):
    candidates = random.sample(population, k)
    return min(candidates, key=lambda ind: ind["Cost"])


- This code is intended to select an individual from the population using the tournament method.

- we randomly select k individuals from the population to form the group of candidates.

- we compare these candidates based on their cost.

- we return the individual with the lowest cost among the candidates.

In [140]:
def crossover(parent1, parent2):
    r1, x1 = parent1["r"], parent1["x"]
    r2, x2 = parent2["r"], parent2["x"]

    point_r = random.randint(1, NB_COMP-1)
    point_x = random.randint(1, NB_VM-1)

    r_child = np.concatenate([r1[:point_r], r2[point_r:]])
    x_child = np.concatenate([x1[:point_x], x2[point_x:]])

    return r_child, x_child


- This code aims to perform a crossover between two parents to generate a child in a genetic algorithm.

- We retrieve the replica vectors r and the VM vectors x from both parents.

- We randomly choose a crossover point for the replicas (r) and another for the VMs (x).

- We build the child by combining parts from both parents:

- Replicas before the crossover point come from parent1, the rest from parent2.
- VMs before the crossover point come from parent1, the rest from parent2.

- We return the child (r_child, x_child) resulting from the crossover.

In [141]:
def mutation(r, x, rate=0.2):
    r2, x2 = r.copy(), x.copy()

    # Mutation réplicas
    for i in range(NB_COMP):
        if random.random() < rate:
            r2[i] = max(1, r2[i] + random.choice([-1, 1]))

    # Mutation VMs
    for j in range(NB_VM):
        if random.random() < rate:
            x2[j] = max(0, x2[j] + random.choice([-1, 1]))

    return r2, x2


- This code is intended to perform a slight mutation on a solution (r, x) in a genetic algorithm.

-  we create copies of the vectors r and x so as not to modify the original solution.

- We go through the replicas r, and for each component, we perform a random mutation with a probability given by 'rate'.

- If a mutation occurs, the number of replicas is increased or decreased by 1, ensuring at least 1 replica.

- We go through the VMs x, and for each type of VM, we perform a random mutation with the same probability.

- If a mutation occurs, the number of VMs is increased or decreased by 1, ensuring at least 0 VMs.

- We return the new mutated solution (r2, x2).

In [142]:
def child(parent1, parent2):
    r, x = crossover(parent1, parent2)
    r, x = mutation(r, x)

    # repair constraint
    r, x = repair(r, x)
    if r is None:
        return child(parent1, parent2)

    return {"r": r, "x": x, "Cost": total_cost(x)}


- This code is designed to generate a child from two parents in a genetic algorithm.

- We perform a crossover to combine the replicas and VMs from both parents.

- We apply a light mutation to introduce diversity into the child.

- We use repair() to ensure the child satisfies all constraints (stability, availability, and resources).

- We recursively call the function if the repair fails to generate a valid child.

- We return a dictionary containing "r" "x" and "cost"

In [143]:
def genetic_algorithm(pop_size= 80, generations=200):
    # Initialisation population
    population = [random_individual() for _ in range(pop_size)]

    best = min(population, key=lambda ind: ind["Cost"])

    for gen in range(generations):
        new_population = []

        nb_elites = max(1, pop_size // 10)
        elites = sorted(population, key=lambda ind: ind["Cost"])[:nb_elites]
        new_population.extend(elites)

        while len(new_population) < pop_size:
            p1 = selection(population)
            p2 = selection(population)
            child_ind = child(p1, p2)
            new_population.append(child_ind)

        population = new_population
        best_gen = min(population, key=lambda ind: ind["Cost"])

        if best_gen["Cost"] < best["Cost"]:
            best = best_gen

        #print(f"Génération {gen+1} | best Cost = {best['Cost']:.4f}")

    return best


- The genetic algorithm is used to find an optimal or near-optimal solution to a complex problem by evolving a population of solutions through selection, crossover, and mutation.

- first, we initialize a population of random solutions with individu_aleatoire().

- We identify the best initial solution based on minimum cost.

-  the process is repeat  for a fixed number of generations.

- We create a new population at each generation.

- We retain the elites (the top 10% best-performing solutions) to maintain the best solutions.

- We fill the rest of the population by generating children: We select two parents with selection() and we generate a child with enfant() and add it to the new population

- We update the population with the new generation.

- We update the global best solution if a lower-cost solution is found in the current generation.

- At the end, return the best solution found

In [144]:
result_ga = genetic_algorithm()

print("\n=== Result GA ===")
print("Cost :", result_ga["Cost"])
print("Replicas :", result_ga["r"])
print("VMs :", result_ga["x"])

y = greedy_placement(result_ga["r"], result_ga["x"])
print("Placement y_ij :\n", y)



=== Result GA ===
Cost : 1.5721999999999998
Replicas : [5 9 6 5]
VMs : [1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0]
Placement y_ij :
 [[1 0 0 0 0 0 0 4 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 4 0 0 5 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 6 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0]]


- This code runs the genetic algorithm with genetic_algorithm() and stores the best solution in resultat_ga.

It displays:

- The total cost of the solution (resultat_ga["cout"]).

- The number of optimal replicas for each component (resultat_ga["r"]).

- The number of optimal VMs of each type (resultat_ga["x"]).

- It calculates the detailed placement of the replicas on the VMs with greedy_placement().

- It displays the y_ij matrix indicating how many instances of each component are assigned to each type of VM.

In [145]:
def simulated_annealing(
        T_init=50,
        T_min=0.1,
        alpha=0.95,
        iterations=60
    ):

    r, x = initial_solution()
    best_cost = total_cost(x)
    best_r, best_x = r.copy(), x.copy()

    current_r, current_x = r.copy(), x.copy()
    current_cost = best_cost

    T = T_init

    while T > T_min:
        for _ in range(iterations):

            if random.random() < 0.85:
                r2, x2 = neighbor(current_r, current_x)
            else:
                x2 = current_x.copy()
                j = random.randint(0, NB_VM-1)
                x2[j] = random.randint(0, DISPO_VM[j])
                r2 = current_r.copy()

            r2, x2 = repair(r2, x2)
            if r2 is None:
                continue

            new_cost = total_cost(x2)
            delta = new_cost - current_cost

            if delta < 0 or random.random() < np.exp(-delta/T):
                current_r, current_x = r2, x2
                current_cost = new_cost

                if new_cost < best_cost:
                    best_cost = new_cost
                    best_r, best_x = r2.copy(), x2.copy()

        T *= alpha

    y = greedy_placement(best_r, best_x)
    return best_r, best_x, y, best_cost


- This code aims to optimize the placement of components on VMs while minimizing the total cost using the Simulated Annealing algorithm.

- We initialize a feasible solution (r, x) with initial_solution() and store the initial best cost.

- We repeat the process while the temperature T is above T_min.

At each temperature, a fixed number of iterations is performed: We generate a neighbor of the current solution to explore the solution space (With 85% probability, a small change using neighbor(), otherwise, a large random jump on the VMs)

- We repair the neighbor solution with repair() to ensure feasibility.

- We calculate the neighbor's cost and the difference delta with the current solution.

- We accept the neighbor if it is better (delta < 0) or with a probability based on the temperature (exp(-delta/T)), allowing the algorithm to escape local minima.

- We update the best solution if the new cost is lower than the current best cost.

- We gradually reduce the temperature T with the factor alpha.

At the end, we compute the final placement of replicas on VMs with greedy_placement() and return:

- best_r: the optimal number of replicas

- best_x: the optimal number of VMs

- y: the placement matrix y_ij

- best_cost: the minimum cost found.

In [146]:
applications = [1, 2, 5, 10]
result = {}

for k in applications:
    print(f"\n=== Optimization for {k} applications ===")

    # Update lambdas
    lamb_k = k * lamb

    # Update the minimum replicas function
    def replicas_minimales():
        return np.ceil(lamb_k / mu).astype(int)

    # ============================
    # HILL CLIMBING
    # ============================
    try:
        r_hc, x_hc, y_hc, cost_hc = hill_climbing()
    except Exception as e:
        print(f"[HC] Error for {k} applications:", e)
        r_hc = x_hc = y_hc = cost_hc = None

    # ============================
    # GENETIC ALGORITHM
    # ============================
    try:
        res_ga = genetic_algorithm()
        r_ga = res_ga["r"]
        x_ga = res_ga["x"]
        cost_ga = res_ga["Cost"]
    except Exception as e:
        print(f"[GA] Error for {k} applications:", e)
        r_ga = x_ga = cost_ga = None

    # ============================
    # SIMULATED ANNEALING
    # ============================
    try:
        r_sa, x_sa, y_sa, cost_sa = simulated_annealing()
    except Exception as e:
        print(f"[SA] Error for {k} applications:", e)
        r_sa = x_sa = y_sa = cost_sa = None

    # Saving the results
    result[k] = {
        "HC_cost": cost_hc,
        "HC_r": r_hc,
        "HC_x": x_hc,

        "GA_cost": cost_ga,
        "GA_r": r_ga,
        "GA_x": x_ga,

        "SA_cost": cost_sa,
        "SA_r": r_sa,
        "SA_x": x_sa
    }

    # Displaying results for this k
    print("  - HC cost :", cost_hc)
    print("  - GA cost :", cost_ga)
    print("  - SA cost :", cost_sa)



=== Optimization for 1 applications ===
  - HC cost : 1.7648
  - GA cost : 1.5721999999999998
  - SA cost : 1.7648

=== Optimization for 2 applications ===
  - HC cost : 2.6471999999999998
  - GA cost : 2.6471999999999998
  - SA cost : 2.6471999999999998

=== Optimization for 5 applications ===
  - HC cost : 7.0592
  - GA cost : 6.4832
  - SA cost : 7.0592

=== Optimization for 10 applications ===
  - HC cost : 13.812000000000001
  - GA cost : 13.1176
  - SA cost : 16.6258


- We test three algorithms (Hill Climbing, Genetic Algorithm, Simulated Annealing) for different application loads.

- We update the minimum number of replicas according to the load.

- We store and display the costs and optimal solutions for each algorithm.

In [147]:
def simulated_annealing(
        T_init=10.0,         # initial temperature
        T_min=0.001,         # minimum temperature
        alpha=0.97,          # cooling rate
        iterations=300       # iteration per temperature
    ):

    # -------- Initial solution--------
    r, x = initial_solution()
    y = greedy_placement(r, x)
    best_r, best_x, best_y = r.copy(), x.copy(), y.copy()
    best_cost = total_cost(x)

    current_r, current_x, current_y = r.copy(), x.copy(), y.copy()
    current_cost = best_cost

    T = T_init

    # -------- Simulated annealing loop --------
    while T > T_min:
        for _ in range(iterations):

            # Generate neighbor
            r2, x2 = neighbor(current_r, current_x)

            # repair solution
            r2, x2 = repair(r2, x2)
            if r2 is None:
                continue

            y2 = greedy_placement(r2, x2)
            if y2 is None:
                continue

            new_cost = total_cost(x2)
            delta = new_cost - current_cost

            # -------- Acceptance Criteria --------
            if delta < 0:
                # best solution --> accept
                current_r, current_x, current_y = r2, x2, y2
                current_cost = new_cost

                # Update best
                if new_cost < best_cost:
                    best_r, best_x, best_y = r2, x2, y2
                    best_cost = new_cost

            else:
                # Accept a bad solution with probability exp(-delta / T)
                prob = np.exp(-delta / T)
                if random.random() < prob:
                    current_r, current_x, current_y = r2, x2, y2
                    current_cost = new_cost

        # cooling
        T *= alpha

    return best_r, best_x, best_y, best_cost


- Simulated annealing is used to optimize the placement of replicas on VMs

- Neighbors of the current solution are generated and repaired to ensure feasibility.

- Better solutions are accepted, and sometimes worse solutions are accepted based on the probability related to the temperature.

- The temperature is gradually reduced until reaching T_min.

- The best solution found is returned along with its cost

In [148]:
r_sa, x_sa, y_sa, cost_sa = simulated_annealing()

print("=== Résultat Simulated Annealing ===")
print("cost :", cost_sa)
print("Réplicas :", r_sa)
print("VMs :", x_sa)
print("Placement y_ij :\n", y_sa)


=== Résultat Simulated Annealing ===
cost : 13.1176
Réplicas : [41 83 58 46]
VMs : [ 0  0  0  0  1  0  0  8  0  0 10  0  0  0  0  0  0]
Placement y_ij :
 [[ 0  0  0  0  4  0  0 37  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 27  0  0 56  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 58  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 46  0  0  0  0  0  0]]


- We run the Simulated Annealing algorithm to find an optimized solution
- We retrieve the best solution: number of replicas r_sa, number of VMs x_sa, placement y_sa, and cost cost_sa
- We display the total cost, the optimal replicas, the VMs used, and the placement matrix y_ij.